In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from import_deps import PyModule, ModuleSet
import pygraphviz

import pytrans
pytrans_module_root = Path(pytrans.__file__).resolve().parent


In [3]:
pytrans_module_root

WindowsPath('D:/Scratch/pytrans-examples/deps/pytrans/pytrans')

In [4]:


def get_imports(pkg_modules, module_path):
    module = pkg_modules.by_path[module_path]
    imports = pkg_modules.get_imports(module, return_fqn=True)
    return list(sorted(imports))


def _imports(module_root):
    """find imports from a python module"""
    base_path = Path(module_root)
    pkg_modules = ModuleSet(base_path.glob('**/*.py'))
    imports = {}
    for _, module in pkg_modules.by_name.items():
        importname = '.'.join(module.fqn)
        imports[importname] = get_imports(pkg_modules, module.path)
    return imports

imports = _imports(pytrans_module_root)



In [6]:


graph = pygraphviz.AGraph(strict=False, directed=True)
graph.node_attr['style'] = 'filled'
graph.node_attr['color'] = 'lightblue2'
for source, sinks in imports.items():
    if source.endswith('__init__'):
        color = 'lightyellow'
        source = source[:-9]
    else:
        color = 'lightblue2'
    for sink in sinks:
        graph.add_edge(source, sink)
        node = graph.get_node(source)
        node.attr['color'] = color
    
graph.layout(prog="twopi")  # , args="-Nshape=box -Efontsize=6")

dotfile = 'pytrans.dot'
graph.write(dotfile)

graph.draw('figures/fig_imports.png')
